# SNS 데이터 각각의 일자별 뉴스기사 개수 구하기

In [1]:
# 제공된 SNS 데이터 파일 불러오기

import glob
import os
import pandas as pd
from konlpy.tag import Okt

okt = Okt()
os.getcwd()

filenames = glob.glob('../data/SNS/*.xlsx')

In [2]:
# SNS 데이터 파일 리스트로
filenames

['../data/SNS\\SNS_1.xlsx',
 '../data/SNS\\SNS_2.xlsx',
 '../data/SNS\\SNS_3.xlsx',
 '../data/SNS\\SNS_4.xlsx',
 '../data/SNS\\SNS_5.xlsx',
 '../data/SNS\\SNS_6.xlsx',
 '../data/SNS\\SNS_7.xlsx',
 '../data/SNS\\SNS_8.xlsx']

In [3]:
# SNS 데이터 하나씩 불러와서 for문

for i in range(len(filenames)):
    sns = pd.read_excel(filenames[i])
    
    # 뉴스만 사용
    sns = sns[sns.SECTION == '뉴스']
    
    # sns 각각의 형태소 담는 리스트
    snsmorph = []
    
    for j in range(len(sns)) :
        snsmorph.append(okt.morphs(sns['TITLE'].iloc[j]))
     
    # sns의 열로 형태소 리스트 추가
    sns['MORPH_T1'] = snsmorph
    
    # 리스트 안의 원소들을 ,(쉼표) 기준으로 분해
    sns2 = sns['MORPH_T1']
    sns3 = [','.join(j) for j in sns2]
    sns['MORPH_T'] = sns3
    
    # 추후 계산 작업을 위해 날짜 데이터 타입을 str으로 변경
    snsdate = list(sns['DATE'])
    snsdate = [str(snsdate[j])[:8] for j in range(len(snsdate))]
    sns['DATE'] = snsdate 
    
    # sns 형태소 분석 결과에서 미세먼지 있을 경우 1, 아닐 경우 0으로 카운트 -> 열로 추가
    snscount = []
    for w in range(len(sns)) :
        snsw = sns['MORPH_T1'].iloc[w]
        if '미세먼지' in snsw :
            snscount.append('1')
        elif '초미세먼지' in snsw :
            snscount.append('1')
        else :
            snscount.append('0')
    sns['COUNT'] = snscount
    
    # 합계 계산할 수 있도록 집계 개수 데이터 타입을 int로 변경
    snsdc = sns[['DATE','COUNT']]
    co = [int(snsdc['COUNT'].iloc[j]) for j in range(len(snsdc))]
    snsdc['COUNT']=co
    
    # 날짜별 합계 구하기
    groups = snsdc.groupby(snsdc.DATE)
    gr = groups.sum()
    
    # 날짜가 인덱스로 되어 있으므로 열로 분리
    grl = list(gr.index)
    gr['DATE'] = grl
    
    # 인덱스 초기화 후 열 순서 변경
    gr = gr.reset_index(drop=True)
    gr = gr[['DATE', 'COUNT']]

    gr.to_csv('../data/SNS/count/countsnstitle'+str(i+1)+'.csv',index=False)
    
    print(str(i+1)+'번째 파일 완료')
    print('-------------------------------------')

1번째 파일 완료
-------------------------------------


C:\Anaconda3\lib\site-packages\ipykernel_launcher.py:43: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


2번째 파일 완료
-------------------------------------
3번째 파일 완료
-------------------------------------
4번째 파일 완료
-------------------------------------
5번째 파일 완료
-------------------------------------
6번째 파일 완료
-------------------------------------
7번째 파일 완료
-------------------------------------
8번째 파일 완료
-------------------------------------


# 일자별 뉴스기사 개수 하나의 파일로 합치기 

In [4]:
import glob
import os
import pandas as pd

os.getcwd()

filenames = glob.glob('../data/SNS/count/*.csv')

In [5]:
filenames

['../data/SNS/count\\countsnstitle1.csv',
 '../data/SNS/count\\countsnstitle2.csv',
 '../data/SNS/count\\countsnstitle3.csv',
 '../data/SNS/count\\countsnstitle4.csv',
 '../data/SNS/count\\countsnstitle5.csv',
 '../data/SNS/count\\countsnstitle6.csv',
 '../data/SNS/count\\countsnstitle7.csv',
 '../data/SNS/count\\countsnstitle8.csv']

In [6]:
# 처음 if문에서 0번째, 1번째 파일을 써야하므로 전체 개수에서 -1을 해서 for문 돌린다.
for i in range(len(filenames) - 1):

    # merge가 진행될 기본적인 데이터프레임 만들기
    if i == 0:
        f1 = pd.read_csv(filenames[0])
        f2 = pd.read_csv(filenames[1])
        
        # 길이에 따라 cc, dd 정의
        if len(f1) >= len(f2):
            cc = f1
            dd = f2
        else:
            dd = f1
            cc = f2
        
        # merge 위치 결정 후 합치는 작업
        results = cc.merge(dd, on='DATE', how='outer')
        print('first')

    else:

        f = pd.read_csv(filenames[i + 1])
        
        if len(f) >= len(results):
            cc = f
            dd = results
        else:
            dd = f
            cc = results
        
        # 파일 불러와서 차례대로 합치는 작업 (results에 누적)
        results = cc.merge(dd, on='DATE', how='outer')
        print(str(i+1))

results.to_csv('../data/COUNT_T_MERGE.csv', index = False)

first
2
3
4
5
6
7


# 뉴스기사 개수 일자별 최종 합계

In [7]:
cnt = pd.read_csv('../data/COUNT_T_MERGE.csv')

# NaN값 0으로 채우기
cnt = cnt.fillna(0)

# 일자별로 뉴스개수 더하기
cnt['COUNT_T'] = cnt['COUNT_x']+cnt['COUNT_y']+cnt['COUNT_x.1']+cnt['COUNT_y.1']+cnt['COUNT_x.2']+cnt['COUNT_y.2']+cnt['COUNT_x.3']+cnt['COUNT_y.3']

# DATE 기준 정렬
cnt = cnt.sort_values(by='DATE')

cnt.to_csv('../data/COUNT_T_MERGE2.csv',index=False)